In [1]:
import pandas as pd
import numpy as np
'''
data preprocessing
'''

#주가데이터 파일을 로드
csv_data = pd.read_csv("./data/005930_200325.csv")
csv_data

,date,start,high,low,end,volume,foreign,person,company,wondollar
0,2020.3.25,48950,49600,47150,48650,52735922,-2751527,4980190,-1695448,1229.9
1,2020.3.24,43850,46950,43050,46950,49801908,2929769,-6174614,3254324,1249.6
2,2020.3.23,42600,43550,42400,42500,41701626,-8143293,10988297,-3248991,1266.5
3,2020.3.20,44150,45500,43550,45400,49730008,-5133212,3248749,1669124,1246.5
4,2020.3.19,46400,46650,42300,42950,56925513,-5807616,4105594,1612440,1285.7
...,...,...,...,...,...,...,...,...,...,...
592,2017.10.23,54600,54640,54000,54300,8311050,30468,4568,-63065,1130.2
593,2017.10.20,52800,54100,52800,53840,8027050,29906,-2283,-39735,1131.0
594,2017.10.19,54700,54700,52980,52980,12108700,-19892,46171,-46270,1132.4
595,2017.10.18,54820,55240,54040,54760,10110750,8951,-3285,-23873,1129.9


In [2]:
#필요한 부분을 가져온다
stdata = csv_data[["start","high", "low","end","volume","foreign", "person","company","wondollar"]]
stdata.head()

#전일대비 가격 변화율을 추가한다
price_change = -np.diff(stdata["end"].to_numpy())
price = stdata["end"].to_numpy()
price = price[1:]
price_change_rate = (price_change/price)*100
price_change_rate=np.append(price_change_rate,[0])
stdata['price_change_rate'] = price_change_rate

#전일대비 거래량 변화율을 추가한다


#캔들 (종가,시가,고가,저가) 관련 정보를 추가한다
#시가-종가
start = stdata["start"].to_numpy()
end = stdata["end"].to_numpy()
candle_end_start = end-start
stdata['candle_end_start'] = candle_end_start

#고가-저가 (당일변동폭) 
high = stdata["high"].to_numpy()
low = stdata["low"].to_numpy()
candle_high_low = high-low
stdata['candle_high_low'] = candle_high_low


#상승or하락 표시 상승:1 보함:0 하락:-1
up_or_down = np.zeros(597)

price_change_rate = stdata["price_change_rate"].to_numpy()
for idx, val in enumerate(price_change_rate):
    if val>0:
        up_or_down[idx]=1
    if val<0:
        up_or_down[idx]=-1
stdata['up_or_down'] = up_or_down

stdata


,start,high,low,end,volume,foreign,person,company,wondollar,price_change_rate,candle_end_start,candle_high_low,up_or_down
0,48950,49600,47150,48650,52735922,-2751527,4980190,-1695448,1229.9,3.620873,-300,2450,1.0
1,43850,46950,43050,46950,49801908,2929769,-6174614,3254324,1249.6,10.470588,3100,3900,1.0
2,42600,43550,42400,42500,41701626,-8143293,10988297,-3248991,1266.5,-6.387665,-100,1150,-1.0
3,44150,45500,43550,45400,49730008,-5133212,3248749,1669124,1246.5,5.704307,1250,1950,1.0
4,46400,46650,42300,42950,56925513,-5807616,4105594,1612440,1285.7,-5.811404,-3450,4350,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,54600,54640,54000,54300,8311050,30468,4568,-63065,1130.2,0.854383,-300,640,1.0
593,52800,54100,52800,53840,8027050,29906,-2283,-39735,1131.0,1.623254,1040,1300,1.0
594,54700,54700,52980,52980,12108700,-19892,46171,-46270,1132.4,-3.250548,-1720,1720,-1.0
595,54820,55240,54040,54760,10110750,8951,-3285,-23873,1129.9,-0.072993,-60,1200,-1.0


In [3]:
#정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
fitted = scaler.fit(stdata)
stdata_arr = scaler.transform(stdata)
stdata_arr.shape

(597, 13)

In [4]:
#정답1 : 가격변화강도 = 가격 변화율 절대값
price_change = -np.diff(stdata["end"].to_numpy())

price = stdata["end"].to_numpy()
price = price[1:]

price_change_rate = (price_change/price)*100
price_change_power = abs(price_change_rate)

price_change_power.shape
#3.62087327, 10.47058824,  6.3876652 ,  5.70430733,

(596,)

In [5]:
#정답2 : 상승(변화율>0) or 하락 
#상승일경우 1 하락일경우 0

up_or_down = np.zeros(596)

price_change = -np.diff(stdata["end"].to_numpy())
for idx, val in enumerate(price_change):
    if val>0:
        up_or_down[idx]=1

up_or_down.shape
#1., 1., 0., 1., 0., 0., 0., 0

(596,)

In [6]:
#inputdata1 : 15일 짜리 데이터 생성 

#0행은 버린다 (정답이없기떄문)
#596일의 데이터로 15일짜리 데이터를 생성하면 596-15+1=582 개의 데이터가 생성된다
stdata_arr=stdata_arr[1:]
stdata_arr

input_data_count = stdata_arr.shape[0]-15+1


#596*13열     [[13개], 
#             [13개], 
#             [13개],
#             [13개],
#              ... ]
#
# 
#              을 [    [ [13개],[13개],[13개]..........[13개] (13개짜리 15개) ],
#                [ [13개],[13개],[13개]..........[13개] (13개짜리 15개) ],
#                [ [13개],[13개],[13개]..........[13개] (13개짜리 15개) ],
#               .
#               .
#               . 
#               582개 
#    
#            ]  (582,15,13)    형태로 만들어야 한다

#for row in stdata_arr:
#    for elem in row 


    

In [7]:
inputdata1 = np.empty((582,15,13), dtype=np.float64)

for i in range(input_data_count):
    for j in range(15):
        inputdata1[i][j]=stdata_arr[i+j]

In [8]:
inputdata1.shape

(582, 15, 13)

In [9]:
#pytorch cross entry loss 에서는 target(정답)을 one hot encode 허자 얺음
#from sklearn import preprocessing

#(하락,상승)으로 one hot encoding
#
#up_or_down=up_or_down.astype(np.int64)
#num = np.unique(up_or_down, axis=0)
#num = num.shape[0]

#answer1 = np.eye(num)[up_or_down]
#answer1 = answer1[:582]
#answer1.shape

In [10]:
up_or_down=up_or_down.astype(np.int)
up_or_down=up_or_down[:582]
up_or_down = up_or_down.reshape(-1,1)
up_or_down.shape

(582, 1)

In [11]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [12]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [13]:
#train : 502 test : 80
inputdata1_train = inputdata1[:502]
inputdata1_test = inputdata1[502:]

up_or_down_train = up_or_down[:502]
up_or_down_test = up_or_down[502:]

In [14]:
class TrainDataset(Dataset):
  def __init__(self):
    self.x_data = inputdata1_train
    self.y_data = up_or_down_train

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.tensor(self.x_data[idx])
    y = torch.tensor(self.y_data[idx])
    return x, y

In [15]:
class TestDataset(Dataset):
  def __init__(self):
    self.x_data = inputdata1_test
    self.y_data = up_or_down_test

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.tensor(self.x_data[idx])
    y = torch.tensor(self.y_data[idx])
    return x, y

In [16]:
train_dataset = TrainDataset()
test_dataset = TestDataset()
dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)


In [17]:
# feature 갯수
input_size = 13

#output from the cell. 바로 결과 예측을 위해서 2로 설정 
hidden_dim = 1

batch_size = 1 

# 15일
sequence_length = 15

# one layer rnn이다.
n_layers = 1 

#inputs = (batch_size * seq_len * input_size ) 인 행렬\
#hidden = (num_layers * batch_size * hidden_size ) 인 행렬
#output = (batch,seq_len,hidden)

In [18]:
#상승 또는 하락 판별 모델
class Model(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.GRU(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        #self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        #out = out.contiguous().view(-1, self.hidden_dim)
        #out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return hidden

In [72]:
#상승 또는 하락 판별 모델
class Model(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        #self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        h0 = self.init_hidden(batch_size)
        c0 = self.init_hidden(batch_size)
        

        # Passing in the input and hidden state into the model and obtaining outputs
        out, (hn,cn) = self.rnn(x, (h0,c0))
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        #out = out.contiguous().view(-1, self.hidden_dim)
        #out = self.fc(out)
        
        return out,
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.rand(self.n_layers, batch_size, self.hidden_dim)
        hidden = nn.init.xavier_normal_(hidden).to(device)
        return hidden

In [77]:
#모델 초기화
model = Model(input_size=13, hidden_dim=1, n_layers=3)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
nb_epochs = 20
lr=0.005

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [78]:
#학습
#for epoch in range(nb_epochs + 1):
#     for batch_idx, samples in enumerate(dataloader):
for epoch in range(nb_epochs + 1):
    print("epoch: ", epoch)
    for batch_idx, (input_data,label) in enumerate(dataloader):
        optimizer.zero_grad()
        input_data = input_data.float().to(device)
        label = label.long().to(device)
        output, = model(input_data)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        if(batch_idx%10==0):
            print("epoch: ",epoch, "  iter:",batch_idx,"/502")
            print("Loss: {:.4f}".format(loss.item()))
            print("")
        

epoch:  0
epoch:  0   iter: 0 /502
Loss: 2.6571

epoch:  0   iter: 10 /502
Loss: 2.4346

epoch:  0   iter: 20 /502
Loss: 2.6396

epoch:  0   iter: 30 /502
Loss: 2.6470

epoch:  0   iter: 40 /502
Loss: 2.6773

epoch:  0   iter: 50 /502
Loss: 2.4878

epoch:  0   iter: 60 /502
Loss: 2.4947

epoch:  0   iter: 70 /502
Loss: 2.5672

epoch:  0   iter: 80 /502
Loss: 2.5059

epoch:  0   iter: 90 /502
Loss: 2.4564

epoch:  0   iter: 100 /502
Loss: 2.7719

epoch:  0   iter: 110 /502
Loss: 2.4840

epoch:  0   iter: 120 /502
Loss: 2.3883

epoch:  0   iter: 130 /502
Loss: 2.7157

epoch:  0   iter: 140 /502
Loss: 2.5230

epoch:  0   iter: 150 /502
Loss: 2.0872

epoch:  0   iter: 160 /502
Loss: 2.6520

epoch:  0   iter: 170 /502
Loss: 2.1174

epoch:  0   iter: 180 /502
Loss: 1.9034

epoch:  0   iter: 190 /502
Loss: 2.3076

epoch:  0   iter: 200 /502
Loss: 2.0202

epoch:  0   iter: 210 /502
Loss: 2.0299

epoch:  0   iter: 220 /502
Loss: 2.1579

epoch:  0   iter: 230 /502
Loss: 1.9496

epoch:  0   iter:

KeyboardInterrupt: 

In [71]:
hidden = torch.rand(4,3,2)
hidden

tensor([[[0.9554, 0.1795],
         [0.5039, 0.5256],
         [0.8298, 0.8800]],

        [[0.9116, 0.8294],
         [0.1725, 0.4755],
         [0.3395, 0.5084]],

        [[0.4154, 0.0935],
         [0.5873, 0.8907],
         [0.6696, 0.3904]],

        [[0.9994, 0.5405],
         [0.5444, 0.4122],
         [0.4245, 0.6817]]])